# Orders of magnitude

This short chapter is about **thinking in scales**.

You will learn how to:
- write numbers in **scientific notation**,
- compare quantities by **factors of ten** (“within an order of magnitude”),
- use **base-10 logarithms** as a *scale*,
- do quick back-of-the-envelope computations

## 1) Why orders of magnitude?

Many variables in ecological economics and sustainability science span large ranges:
- GDP per capita (thousands to tens of thousands of USD),
- population (thousands to billions),
- emissions (per person or per country) (kg to Gt).
- national energy consumption (PJ/year),
- national material use (tons/year)

If we plot or compare these values on a normal (linear) scale, the small stuff disappears.
A **logarithmic scale** fixes this by turning multiplicative gaps (×10, ×100, ×1000) into additive steps (+1, +2, +3).

## 2) Scientific notation

Any positive number can be written as

```{math}
n = a\times 10^b,
```

where:
- the exponent **`b`** is an integer, and
- the mantissa **`a`** is a number around 1 (for example between 1 and 10).

Examples:
- 3,200 = 3.2 × 10³  
- 0.007 = 7 × 10⁻³  
- 81,000,000 = 8.1 × 10⁷

This representation makes scale comparisons easy: the exponent `b` tells you roughly “how many zeros.”


## 3) “Within an order of magnitude” = “within a factor of 10”

Two positive numbers `x` and `y` are **within an order of magnitude** if the larger is **less than 10 times** the smaller:

So:
- 2 and 9 are within an order of magnitude (9/2 = 4.5),
- 2 and 200 are *not* (200/2 = 100).
- 2 and 20 are also not *not* for the purpose of this book. Some conventions would say this is within one order of magnitude, but we strictly say this only if it is less than a ten times difference. 

The exact boundary convention matters less than the **habit**: asking “is this a ×2 difference, ×10, ×100?”.


## Mini-exercise: scientific notation with energy use

Suppose (roughly) that annual **energy use per person** is:

- USA: **300 GJ/person/year**
- Nigeria: **10 GJ/person/year**

1. Write each value in scientific notation  $a\times 10^b$ with  $1 \le a < 10$.  
2. Compare the two values by a **ratio**. About how many “times larger” is the USA value?  
3. Is that difference **within one order of magnitude** (a factor of 10), or not?

<details>
<summary><b>Solution (click to expand)</b></summary>

### 1) Scientific notation

- USA:  
 $300 = 3.0 \times 10^{2}$

  so \(a=3.0\), \(b=2\).

- Nigeria:  
  $10 = 1.0 \times 10^{1}$
  so \(a=1.0\), \(b=1\).

### 2) Ratio (times larger)

$\frac{300}{10} = 30$

So the USA value is about **30× larger**.

### 3) Order-of-magnitude check

A factor of 10 corresponds to **one order of magnitude**.  
Since $30 = 3\times 10$, the difference is **a bit more than one order of magnitude**  
(about $(\log_{10}(30)\approx 1.48\)$ orders of magnitude).
</details>



## 3.1) Logarithms


```{admonition} Logarithms for dummies (base 10)
:class: tip

A **logarithm** is just the *inverse* of exponentiation.

### What does {math}`\log_{10}(x)` mean?
It answers:

> **“Which power of 10 gives me `x`?”**

So if {math}`10^b = x`, then

```{math}
\log_{10}(x) = b.
```

### A note on the notation (so it stops looking like alien poetry)
- The subscript **10** in {math}`\log_{10}` tells you the **base**.  
  So {math}`\log_{10}` means “log base 10”.
- The input in parentheses, the **argument** {math}`x`, is the number you’re taking the log of.
- In many books you’ll also see:
  - {math}`\ln(x)` which means the **natural log** (base {math}`e \approx 2.718`)
  - sometimes just {math}`\log(x)` with the base left implicit (annoying, but common).
- In this chapter, we use **base 10** because it matches “orders of magnitude” (powers of ten).

Equivalently, this is the key “inverse” relationship:

```{math}
\log_{10}(10^b)=b.
```

Quick examples:
- {math}`\log_{10}(1)=0` because {math}`10^0=1`
- {math}`\log_{10}(10)=1` because {math}`10^1=10`
- {math}`\log_{10}(1000)=3` because {math}`10^3=1000`
- {math}`\log_{10}(0.01)=-2` because {math}`10^{-2}=0.01`

### Why logs are perfect for “orders of magnitude”
A factor of 10 becomes “+1” on the log scale:
- multiplying by 10 adds 1 to {math}`\log_{10}`
- dividing by 10 subtracts 1

That’s why this identity is so useful:

```{math}
\log_{10}(x/y)=\log_{10}(x)-\log_{10}(y).
```

So the “scale ruler” for orders of magnitude is:

```{math}
\Delta \text{orders} \approx \log_{10}(x) - \log_{10}(y) = \log_{10}(x/y).
```

### Example
If {math}`x/y = 1000`, then

```{math}
\log_{10}(1000)=3,
```

so `x` is **3 orders of magnitude** larger than `y` (i.e., about **1000×**).
```


## 4) Logarithms as a scale ruler

The base-10 logarithm answers:

“What power of 10 gives this number?”

```{math}
\log_{10}(10^b)=b.
```

And the key identity:

```{math}
\log_{10}(x/y)=\log_{10}(x)-\log_{10}(y).
```

So the **difference in orders of magnitude** between `x` and `y` is basically:

```{math}
\Delta \text{orders} \approx \log_{10}(x) - \log_{10}(y) = \log_{10}(x/y).
```

Example: if `x/y = 1000`, then `log10(1000) = 3` → a **3-order-of-magnitude** difference.


## 5) Mini exercise (with solution)

To remain and get more and more into coding habits. Try to write a python function that conducts the above scale ruler computation. This is not super important for our proceeding chapters but practice is never mistaken.

For this you also need to import the math package. Just do so via 

```python
import math
```


```{admonition} Hints
:class: tip

- **Logarithms need positive inputs**: before computing anything, check that `x > 0` and `y > 0`.
  If not, raise a `ValueError` with a helpful message.
- Convert inputs to numbers you can safely divide: use `x = float(x)` and `y = float(y)`.
  This also helps if someone passes strings like `"300"` or integers.
- The core computation is just: `math.log10(x / y)`.


```{dropdown} Solution (Python)
```python
import math

def orders_apart(x, y):
    """
    Return the difference in orders of magnitude between x and y:
    log10(x/y). Requires x>0 and y>0.
    """
    x = float(x)
    y = float(y)
    if x <= 0 or y <= 0:
        raise ValueError("orders_apart requires positive x and y.")
    return math.log10(x / y)

## 6) Takeaways

- An **order of magnitude** is a factor of ~10.
- Scientific notation separates **scale** (`10^b`) from **detail** (`a`).
- Logarithms turn **multiplicative** differences into **additive** differences.

In the next chapter we are going to learn about static scaling relationships in Ecological Economics and pick up some of the most important concepts in all of ecological economics (and sometimes all of science) along the way - Log–log plots, A first look at basic regression analysis and elasticities.


## 6) Extras - a little more math

- include extra math view points on orders of magnitude specifically the geometric mean that defines the boundary between two